In [25]:
import pandas as pd
from collections import Counter
import re
from string import digits

In [26]:
def build_vocab(sentences,threshold):
    counter=Counter()
    for line in sentences:
        #tokenize sentence into words
        token=line.split()
        #lowercase words in list
        low_word=[word.lower() for word in token]
        #remove digits from the list
        remove_digits = str.maketrans('', '', digits)
        word_list= [i.translate(remove_digits) for i in low_word]
        #remove empty string elements from the list
        remove_space=list(filter(None,word_list))
        #update values in counter
        counter.update(remove_space)
    #remove words from vocabulary if frequency is less than given threshold    
    vocabulary=[key for key,value in counter.items() if value>=threshold]    
        
    return vocabulary
        

In [27]:
pd_dataset=pd.read_csv("Flickr8k/captions/train_dataset.txt",delimiter="\t")
pd_frame=pd_dataset.values
print(pd_frame.shape)

(30000, 2)


In [28]:
sentences=[]
for sent in range(pd_frame.shape[0]):
    sentence=pd_frame[sent][1]
    sentences.append(sentence)#append the sentence in list "sentences"
len(sentences)   

30000

In [30]:
vocabulary=build_vocab(sentences,1)
#print(vocabulary)
len(vocabulary)

7668

In [32]:
#vectorization
word2id={value:index for index,value in enumerate(vocabulary)}
id2word={index:value for index,value in enumerate(vocabulary)}

In [59]:
word2id[id2word.get(0)]=7668
word2id["PAD"]=0

id2word[7668]=id2word.get(0)
id2word[0]="PAD"

In [64]:
vocab_size=len(word2id)